**Based on Peter Ferguson's notebook comparing EUCLID and ComCam data (/sdf/data/rubin/user/pferguso/projects/euclid/notebooks/euclid_star_compare.ipynb).**

## 1. User Input

See https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs for the latest values.

In [1]:
repo="/repo/main"
collection="LSSTComCam/runs/DRP/DP1/w_2025_10/DM-49359"
instrument = 'LSSTComCam'
skymap = 'lsst_cells_v1'

## ComCam data
#collection='LSSTComCam/DP1'
#repo = 'dp1'
#skymap = 'lsst_cells_v1'
#instrument = 'LSSTComCam'

#outputFileName = 'comcam_stars_all.w_2025_10.DM-49359.csv'
#outputFileName = 'comcam_stars_all.LSSTComCam_DP1.csv'
outputFileName = 'comcam_stars_all.w_2025_10.DM-49359a.csv'


# From Slide 9 of https://docs.google.com/presentation/d/1NGzrT4t6wDGQ2-2a8rjioToquhx2vOP_KJTrPiCrDDY/edit#slide=id.g33de3f5c849_6_250
tract_list = [453, 454, 4849, 5063, 4848, 2394, 2234, 4016, 4017, 4218, 4217, 5525, 5526, 7611, 7610, 7850, 10463, 10464, 10704]
tract_dict={453: '47 Tuc', 
            454: '47 Tuc',
           4849: 'ECDFS', 
           5063: 'ECDFS',
           4848: 'ECDFS', 
           2394: 'EDFS', 
           2234: 'EDFS',
           4016: 'Fornax', 
           4017: 'Fornax', 
           4218: 'Fornax', 
           4217: 'Fornax', 
           5525: 'Rubin_SV_095-25', 
           5526: 'Rubin_SV_095-25', 
           7611: 'Seagull', 
           7610: 'Seagull', 
           7850: 'Seagull',
           10463: 'Rubin_SV_38_7', 
           10464: 'Rubin_SV_38_7', 
           10704: 'Rubin_SV_38_7'
           }

## 2. Imports

(Many of these imports can be deleted.)

In [2]:
# General python packages
import pylab as plt
import gc
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

# LSST Science Pipelines (Stack) packages
import lsst.daf.butler as dafButler
#import lsst.afw.display as afwDisplay

#import lsst.pex.config as pexConfig
#import lsst.pipe.base as pipeBase

#from lsst.verify import Measurement
#from astropy import units as u

#from lsst.pex.config import Field, ListField

#import datetime
#import logging

#from lsst.analysis.tools.interfaces import MetricMeasurementBundle

#import lsst.geom as geom
#from lsst.afw.image import Image
#from lsst.afw.geom import makeSkyWcs

#import healpy as hp


## 3. Useful functions

In [3]:
# Useful class to stop "Run All" at a cell 
#  containing the command "raise StopExecution"
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

In [4]:
zeropoint = 31.4 # AB zeropoint
def flux2mag(flux):
    return -2.5*np.log10(flux) + zeropoint

## 4.  Main code

In [5]:
# Create two butlers, one including the skymap.
#  (Creating two should not be necessary, but...)
butler = dafButler.Butler(repo, collections=collection)
skybutler = dafButler.Butler(repo, collections=collection, skymap=skymap)

In [6]:
# Define which of the nearly 1000 columns to download from the ComCam ObjectTable...

INCOLS = [
    'coord_ra',
    'coord_dec',
    'detect_isPrimary',   
]
bands="ugrizy"
for band in bands:
    INCOLS += [
        f'{band}_psfFlux',
        f'{band}_psfFluxErr',
        f'{band}_ap12Flux',
        f'{band}_ap12FluxErr',
        f'{band}_extendedness',
        f'{band}_psfFlux_flag'
    ]


In [7]:
comcam_stars_list = []

for tractId in tract_list:

    print(tractId, tract_dict[tractId])

    try:
    
        raw_comcam = butler.get('objectTable_tract', dataId={'skymap': 'lsst_cells_v1', 'tract': tractId}, 
                                collections=[collection],
                                parameters={"columns":INCOLS})

        # Insert tractId as the first column
        raw_comcam.insert(0, 'tractId', tractId)  
    
        # Insert field name -- if known -- as the second column
        if tractId in tract_dict:
            field = tract_dict[tractId]
        else:
            field = 'unknown'
        raw_comcam.insert(1, 'field', field)  

        # Clean the catalog
        sel  = (raw_comcam['detect_isPrimary'] == True)
        sel &= (raw_comcam['r_psfFlux']/raw_comcam['r_psfFluxErr'] > 5)
        for band in ['g','r','i']:
            sel &= (raw_comcam[f'{band}_psfFlux_flag'] == 0)

        comcam = raw_comcam[sel]

        # Find just the (most likely) stars...
        sel_comcam_stars = (comcam['g_extendedness'] < 0.5) & (comcam['r_extendedness'] < 0.5)
        comcam_stars = comcam[sel_comcam_stars] 
        print(f"Number of objects: {len(comcam)}")
        print(f"Number of stars: {len(comcam_stars)}")

        # Append the dataframe to the list
        comcam_stars_list.append(comcam_stars) 

    # Catch any exception
    except Exception as e:

        print(f"An error occurred for tractId {tractId}: {e}")


# Concatenate all dataframes in the list
comcam_stars_all = pd.concat(comcam_stars_list, ignore_index=True)  
print(f"Total number of stars: {len(comcam_stars_all)}")

453 47 Tuc
Number of objects: 50145
Number of stars: 9881
454 47 Tuc
Number of objects: 1049
Number of stars: 115
4849 ECDFS
Number of objects: 59035
Number of stars: 2414
5063 ECDFS
Number of objects: 207011
Number of stars: 7931
4848 ECDFS
Number of objects: 32383
Number of stars: 1364
2394 EDFS
Number of objects: 177207
Number of stars: 7555
2234 EDFS
Number of objects: 21854
Number of stars: 1218
4016 Fornax
Number of objects: 65472
Number of stars: 8230
4017 Fornax
Number of objects: 30140
Number of stars: 2671
4218 Fornax
Number of objects: 19987
Number of stars: 1989
4217 Fornax
Number of objects: 7779
Number of stars: 746
5525 Rubin_SV_095-25
Number of objects: 46349
Number of stars: 5666
5526 Rubin_SV_095-25
Number of objects: 114524
Number of stars: 13785
7611 Seagull
Number of objects: 0
Number of stars: 0
7610 Seagull
Number of objects: 0
Number of stars: 0
7850 Seagull
Number of objects: 0
Number of stars: 0
10463 Rubin_SV_38_7
Number of objects: 135364
Number of stars: 67

In [8]:
# print out the catalog of ComCam stars to the screen...

comcam_stars_all

,tractId,field,coord_ra,coord_dec,detect_isPrimary,u_psfFlux,u_psfFluxErr,u_ap12Flux,u_ap12FluxErr,u_extendedness,...,z_ap12Flux,z_ap12FluxErr,z_extendedness,z_psfFlux_flag,y_psfFlux,y_psfFluxErr,y_ap12Flux,y_ap12FluxErr,y_extendedness,y_psfFlux_flag
0,453,47 Tuc,6.296090,-72.654005,True,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,True,4435.431869,1969.664638,9084.558594,3609.099121,NaN,False
1,453,47 Tuc,6.303631,-72.653795,True,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,True,10111.005646,1969.215678,12901.413086,3603.211670,NaN,False
2,453,47 Tuc,6.302208,-72.650256,True,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,True,5695.485226,1956.217723,7140.451660,3591.461670,NaN,False
3,453,47 Tuc,6.324506,-72.644587,True,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,True,2129.304319,1944.437678,5873.591309,3583.588379,NaN,False
4,453,47 Tuc,6.310524,-72.641722,True,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,True,4507.828534,1937.106355,11522.659180,3572.301758,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76138,10704,Rubin_SV_38_7,37.413064,7.865804,True,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,True
76139,10704,Rubin_SV_38_7,37.414892,7.870120,True,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,True
76140,10704,Rubin_SV_38_7,37.428997,7.892328,True,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,True
76141,10704,Rubin_SV_38_7,37.405061,7.900705,True,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,True


In [9]:
# Output results to a CSV file...

comcam_stars_all.to_csv(outputFileName,index=False)

In [10]:
# Stop here for now...
raise StopExecution

## 5. Sandbox